# Q1)

In [1]:
import numpy as np
import pandas as pd

In [2]:
input = np.array([[2,0,2,1,3,2],
                  [0,2,0,2,2,2],
                  [1,0,1,3,1,1],
                  [0,0,1,1,1,0],
                  [0,1,3,4,1,0],
                  [0,1,0,0,5,2]])
filter = np.array([[1,0,0],
                   [0,1,0],
                   [0,0,1]])

## i) Zero Padding

In [3]:
pad_width=1
z_pad = np.pad(input, pad_width, 'constant')
z_pad

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 2, 0, 2, 1, 3, 2, 0],
       [0, 0, 2, 0, 2, 2, 2, 0],
       [0, 1, 0, 1, 3, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 3, 4, 1, 0, 0],
       [0, 0, 1, 0, 0, 5, 2, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

## ii) Filter

In [4]:
filt_stride = 1
filt_size = int((len(input)-len(filter)+(2*pad_width)) / filt_stride + 1)
filt_size

6

In [5]:
filt = np.zeros((filt_size, filt_size))
filt

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [6]:
for n in range(len(filt[0])):
    for m in range(len(filt)):
        filt[n, m] = np.sum(z_pad[n:n+len(filter), m:m+len(filter)] * filter)

In [7]:
filt

array([[ 4.,  0.,  4.,  3.,  5.,  2.],
       [ 0.,  5.,  3.,  5.,  4.,  5.],
       [ 1.,  1.,  4.,  4.,  3.,  3.],
       [ 1.,  4.,  5.,  3.,  4.,  1.],
       [ 1.,  1.,  3., 10.,  4.,  1.],
       [ 0.,  1.,  1.,  3.,  9.,  3.]])

## iii) Maxpooling

In [8]:
pool_stride = 2
pool_size = 2
output_size = int(len(filt) / pool_size)
output_size

3

In [9]:
output = np.zeros((output_size, output_size))
output

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [10]:
for n in range(len(output[0])):
    for m in range(len(output)):
        output[n, m] = np.max(filt[pool_stride*n:pool_stride*n+pool_size,
                                   pool_stride*m:pool_stride*m+pool_size])

In [11]:
output

array([[ 5.,  5.,  5.],
       [ 4.,  5.,  4.],
       [ 1., 10.,  9.]])

# Q2)

In [12]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [14]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [15]:
#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data', train=True,
                          transform=transforms.ToTensor())
mnist_test = dsets.MNIST(root='MNIST_data', train=False,
                         transform=transforms.ToTensor())

In [16]:
#dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [17]:
#CNN Model (2 conv layers) - 기존 코드
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
            
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)    #flatten
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [18]:
#instantiate CNN Model
model = CNN().to(device)

In [19]:
criterion = torch.nn.CrossEntropyLoss().to(device)    #Softmax is internally computed
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [20]:
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        #image is already size of (28x28), no reshape
        #label is not one-hot coded
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.188491419
[Epoch:    2] cost = 0.0527378693
[Epoch:    3] cost = 0.037320517
[Epoch:    4] cost = 0.0293589365
[Epoch:    5] cost = 0.0242028497
[Epoch:    6] cost = 0.0196823105
[Epoch:    7] cost = 0.0180399958
[Epoch:    8] cost = 0.0133935446
[Epoch:    9] cost = 0.0133590056
[Epoch:   10] cost = 0.0108860331
[Epoch:   11] cost = 0.00962581765
[Epoch:   12] cost = 0.00964104105
[Epoch:   13] cost = 0.0093088001
[Epoch:   14] cost = 0.00876647048
[Epoch:   15] cost = 0.00659862952
Learning Finished!


In [21]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\Users\sec\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\sec\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9889000058174133


In [22]:
#Layer1 kernel size = 4
#Layer3 stride = 2
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=4, stride=1, padding=1),    #kernel_size=4
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),    #stride=2
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        
        self.fc1 = torch.nn.Linear(2 * 2 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)    #flatten
        out = self.layer4(out)
        out = self.fc2(out)
        return out

Layer3 output size = (2, 2, 128)

In [23]:
model = CNN().to(device)

In [24]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.261937171
[Epoch:    2] cost = 0.0666275695
[Epoch:    3] cost = 0.0459767282
[Epoch:    4] cost = 0.0364673473
[Epoch:    5] cost = 0.0291582905
[Epoch:    6] cost = 0.0248196013
[Epoch:    7] cost = 0.0221298952
[Epoch:    8] cost = 0.0176844075
[Epoch:    9] cost = 0.0156269092
[Epoch:   10] cost = 0.0159568079
[Epoch:   11] cost = 0.0113323163
[Epoch:   12] cost = 0.011732325
[Epoch:   13] cost = 0.0116861621
[Epoch:   14] cost = 0.00903153326
[Epoch:   15] cost = 0.00899495278
Learning Finished!


In [26]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\Users\sec\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\sec\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9871000051498413


layer를 추가했음에도 cost는 살짝 증가, accuracy는 살짝 감소<br>
근데 사실 별 큰 차이 없어 보인다